# Experimentation and Uplift Testing

Trial stores were performed in stores 77, 86 and 88

In [2]:
import re
import xlrd
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.patches as mpatches
from datetime import datetime, timedelta
from scipy.stats import ttest_ind, linregress

pd.options.mode.chained_assignment = None  

In [99]:
# customer_data = pd.read_csv('QVI_purchase_behaviour.csv')
# transaction_data = pd.read_excel('QVI_transaction_data.xlsx')

#cleaned transaction_data merged with customer_data
data = pd.read_csv('QVI_data.csv', parse_dates=['DATE']) 

## I. Data Cleaning

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   LYLTY_CARD_NBR    264834 non-null  int64         
 1   DATE              264834 non-null  datetime64[ns]
 2   STORE_NBR         264834 non-null  int64         
 3   TXN_ID            264834 non-null  int64         
 4   PROD_NBR          264834 non-null  int64         
 5   PROD_NAME         264834 non-null  object        
 6   PROD_QTY          264834 non-null  int64         
 7   TOT_SALES         264834 non-null  float64       
 8   PACK_SIZE         264834 non-null  int64         
 9   BRAND             264834 non-null  object        
 10  LIFESTAGE         264834 non-null  object        
 11  PREMIUM_CUSTOMER  264834 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(6), object(4)
memory usage: 24.2+ MB


In [100]:
data['YEARMONTH'] = data.DATE.dt.strftime("%Y%m").astype(int)

In [101]:
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,YEARMONTH
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,201810
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,201809
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,201903
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,201903
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,201811


In [102]:
data[data.STORE_NBR.isin([77,86,88])]['YEARMONTH'].nunique()

12

In [103]:
data[(data.STORE_NBR.isin([77,86,88])) & (data.YEARMONTH < 201902)]['YEARMONTH'].nunique()

7

There are 3 trials which began in Feb 2019. In order to compare with a control group, we would need to have a control group whose pre-trial dates and measures are the most similar to our trial stores.

In [104]:
txn_per_store_month = data.groupby(['STORE_NBR','YEARMONTH'])['TXN_ID'].count().reset_index().groupby('STORE_NBR')['YEARMONTH'].count().reset_index()
possible_stores = txn_per_store_month[(txn_per_store_month.YEARMONTH == 12) & ~(txn_per_store_month.STORE_NBR.isin([77,86,88]))].reset_index(drop=True)

Now, we need to define the measures for our possible control stores. We calculate these measures during the pre-trial date of Feb 2019.
- Monthly overall sales revenue
- Monthly number of customers
- Monthly number of transactions per customer

NOTE that `init_control_data` will contain transaction data of possible control stores during the pre-trial period.

In [107]:
init_stores_lst = possible_stores['STORE_NBR'].to_list()
init_control_data = data[data.STORE_NBR.isin(possible_stores_lst)]
init_control_data.shape[0]

260671

In [108]:
init_cntr_measures = init_control_data.groupby(['STORE_NBR','YEARMONTH'])\
                                      .agg({"TXN_ID":"count",
                                            "TOT_SALES":"sum",
                                            "PROD_QTY":"sum",
                                            "LYLTY_CARD_NBR":"nunique"})\
                                      .reset_index()\
                                      .rename(columns={"TXN_ID":"TXN_COUNT",
                                                       "LYLTY_CARD_NBR":"CUST_COUNT"})
                                                                        
init_cntr_measures['TXN_PER_CUST'] = init_cntr_measures['TXN_COUNT']/init_cntr_measures['CUST_COUNT']
init_cntr_measures['CHPS_PER_CUST'] = init_cntr_measures['PROD_QTY']/init_cntr_measures['CUST_COUNT']
init_cntr_measures['AVE_SPEND'] = init_cntr_measures['TOT_SALES']/init_cntr_measures['PROD_QTY']

init_cntr_measures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3084 entries, 0 to 3083
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STORE_NBR      3084 non-null   int64  
 1   YEARMONTH      3084 non-null   int64  
 2   TXN_COUNT      3084 non-null   int64  
 3   TOT_SALES      3084 non-null   float64
 4   PROD_QTY       3084 non-null   int64  
 5   CUST_COUNT     3084 non-null   int64  
 6   TXN_PER_CUST   3084 non-null   float64
 7   CHPS_PER_CUST  3084 non-null   float64
 8   AVE_SPEND      3084 non-null   float64
dtypes: float64(4), int64(5)
memory usage: 217.0 KB


In [141]:
init_cntr_measures.head()

,STORE_NBR,YEARMONTH,TXN_COUNT,TOT_SALES,PROD_QTY,CUST_COUNT,TXN_PER_CUST,CHPS_PER_CUST,AVE_SPEND
0,1,201807,52,206.9,62,49,1.061224,1.265306,3.337097
1,1,201808,43,176.1,54,42,1.023810,1.285714,3.261111
2,1,201809,62,278.8,75,59,1.050847,1.271186,3.717333
3,1,201810,45,188.1,58,44,1.022727,1.318182,3.243103
4,1,201811,47,192.6,57,46,1.021739,1.239130,3.378947


In [111]:
trial = data[(data.STORE_NBR.isin([77,86,88]))]
trial_measures = trial.groupby(['STORE_NBR','YEARMONTH'])\
                              .agg({"TXN_ID":"count",
                                    "TOT_SALES":"sum",
                                    "PROD_QTY":"sum",
                                    "LYLTY_CARD_NBR":"nunique"})\
                              .reset_index()\
                              .rename(columns={"TXN_ID":"TXN_COUNT",
                                               "LYLTY_CARD_NBR":"CUST_COUNT"})
                                                                        
trial_measures['TXN_PER_CUST'] = trial_measures['TXN_COUNT']/trial_measures['CUST_COUNT']
trial_measures['CHPS_PER_CUST'] = trial_measures['PROD_QTY']/trial_measures['CUST_COUNT']
trial_measures['AVE_SPEND'] = trial_measures['TOT_SALES']/trial_measures['PROD_QTY']

trial_measures

,STORE_NBR,YEARMONTH,TXN_COUNT,TOT_SALES,PROD_QTY,CUST_COUNT,TXN_PER_CUST,CHPS_PER_CUST,AVE_SPEND
0,77,201807,55,296.80,84,51,1.078431,1.647059,3.533333
1,77,201808,48,255.50,74,47,1.021277,1.574468,3.452703
2,77,201809,44,225.20,70,42,1.047619,1.666667,3.217143
3,77,201810,38,204.50,52,37,1.027027,1.405405,3.932692
4,77,201811,44,245.30,67,41,1.073171,1.634146,3.661194
5,77,201812,49,267.30,72,46,1.065217,1.565217,3.712500
6,77,201901,39,204.40,65,35,1.114286,1.857143,3.144615
7,77,201902,45,235.00,74,45,1.000000,1.644444,3.175676
8,77,201903,55,278.50,82,50,1.100000,1.640000,3.396341
9,77,201904,48,263.50,78,47,1.021277,1.659574,3.378205


In [140]:
columns = ['TOT_SALES', 'CUST_COUNT', 'TXN_PER_CUST', 'CHPS_PER_CUST', 'AVE_SPEND']

for i in init_stores_lst:
    trial_store = trial_measures[trial_measures.STORE_NBR==77][columns]
    cntrl_store = init_cntr_measures[init_cntr_measures.STORE_NBR==i][columns]

    corr_measures = trial_store.corrwith(cntrl_store, axis=1, drop=True)
    print(f"FOR STORE_NBR: {i}\n{corr_measures}")

FOR STORE_NBR: 1
0     0.997837
1     0.998508
2     0.999624
3     0.998457
4     0.997294
5     0.998779
6     0.998466
7     0.999176
8     0.998550
9     0.999236
10    0.999731
11    0.996164
dtype: float64
FOR STORE_NBR: 2
Series([], dtype: float64)
FOR STORE_NBR: 3
Series([], dtype: float64)
FOR STORE_NBR: 4
Series([], dtype: float64)
FOR STORE_NBR: 5
Series([], dtype: float64)
FOR STORE_NBR: 6
Series([], dtype: float64)
FOR STORE_NBR: 7
Series([], dtype: float64)
FOR STORE_NBR: 8
Series([], dtype: float64)
FOR STORE_NBR: 9
Series([], dtype: float64)
FOR STORE_NBR: 10
Series([], dtype: float64)
FOR STORE_NBR: 12
Series([], dtype: float64)
FOR STORE_NBR: 13
Series([], dtype: float64)
FOR STORE_NBR: 14
Series([], dtype: float64)
FOR STORE_NBR: 15
Series([], dtype: float64)
FOR STORE_NBR: 16
Series([], dtype: float64)
FOR STORE_NBR: 17
Series([], dtype: float64)
FOR STORE_NBR: 18
Series([], dtype: float64)
FOR STORE_NBR: 19
Series([], dtype: float64)
FOR STORE_NBR: 20
Series([], dt

Series([], dtype: float64)
FOR STORE_NBR: 192
Series([], dtype: float64)
FOR STORE_NBR: 194
Series([], dtype: float64)
FOR STORE_NBR: 195
Series([], dtype: float64)
FOR STORE_NBR: 196
Series([], dtype: float64)
FOR STORE_NBR: 197
Series([], dtype: float64)
FOR STORE_NBR: 198
Series([], dtype: float64)
FOR STORE_NBR: 199
Series([], dtype: float64)
FOR STORE_NBR: 200
Series([], dtype: float64)
FOR STORE_NBR: 201
Series([], dtype: float64)
FOR STORE_NBR: 202
Series([], dtype: float64)
FOR STORE_NBR: 203
Series([], dtype: float64)
FOR STORE_NBR: 204
Series([], dtype: float64)
FOR STORE_NBR: 205
Series([], dtype: float64)
FOR STORE_NBR: 207
Series([], dtype: float64)
FOR STORE_NBR: 208
Series([], dtype: float64)
FOR STORE_NBR: 209
Series([], dtype: float64)
FOR STORE_NBR: 210
Series([], dtype: float64)
FOR STORE_NBR: 212
Series([], dtype: float64)
FOR STORE_NBR: 213
Series([], dtype: float64)
FOR STORE_NBR: 214
Series([], dtype: float64)
FOR STORE_NBR: 215
Series([], dtype: float64)
FOR STO

In [92]:
def correlation(trial_store, cntrl_stores, trial_df=trial_measures, cntrl_df=init_cntr_measures):
    
    for i in cntrl_stores:
        trial_store = trial_df[trial_df.STORE_NBR==trial_store]
        cntrl_store = cntrl_df[cntrl_df.STORE_NBR==i]

        corr_measures = trial_store.corrwith(cntrl_store, method='pearson', axis=0, drop=True)

        if math.isnan(corr_measures.TOT_SALES)==False:
            print(f"FOR STORE_NBR: {i}\n\n{corr_measures[1:]}")

In [132]:
trial_measures[trial_measures.STORE_NBR==77]

,STORE_NBR,YEARMONTH,TXN_COUNT,TOT_SALES,PROD_QTY,CUST_COUNT,TXN_PER_CUST,CHPS_PER_CUST,AVE_SPEND
0,77,201807,55,296.8,84,51,1.078431,1.647059,3.533333
1,77,201808,48,255.5,74,47,1.021277,1.574468,3.452703
2,77,201809,44,225.2,70,42,1.047619,1.666667,3.217143
3,77,201810,38,204.5,52,37,1.027027,1.405405,3.932692
4,77,201811,44,245.3,67,41,1.073171,1.634146,3.661194
5,77,201812,49,267.3,72,46,1.065217,1.565217,3.712500
6,77,201901,39,204.4,65,35,1.114286,1.857143,3.144615
7,77,201902,45,235.0,74,45,1.000000,1.644444,3.175676
8,77,201903,55,278.5,82,50,1.100000,1.640000,3.396341
9,77,201904,48,263.5,78,47,1.021277,1.659574,3.378205


In [133]:
init_cntr_measures[init_cntr_measures.STORE_NBR==1]

,STORE_NBR,YEARMONTH,TXN_COUNT,TOT_SALES,PROD_QTY,CUST_COUNT,TXN_PER_CUST,CHPS_PER_CUST,AVE_SPEND
0,1,201807,52,206.9,62,49,1.061224,1.265306,3.337097
1,1,201808,43,176.1,54,42,1.023810,1.285714,3.261111
2,1,201809,62,278.8,75,59,1.050847,1.271186,3.717333
3,1,201810,45,188.1,58,44,1.022727,1.318182,3.243103
4,1,201811,47,192.6,57,46,1.021739,1.239130,3.378947
5,1,201812,47,189.6,57,42,1.119048,1.357143,3.326316
6,1,201901,36,154.8,42,35,1.028571,1.200000,3.685714
7,1,201902,55,225.4,65,52,1.057692,1.250000,3.467692
8,1,201903,49,192.9,58,45,1.088889,1.288889,3.325862
9,1,201904,43,192.9,57,42,1.023810,1.357143,3.384211


In [143]:
trial_store = trial_measures[trial_measures.STORE_NBR==77][columns]
cntrl_store = init_cntr_measures[init_cntr_measures.STORE_NBR==2][columns]

trial_store.corrwith(cntrl_store, axis=1, drop=True).reset_index(drop=True)

Series([], dtype: float64)

In [147]:
np.corrcoef(trial_store, cntrl_store)[0]

array([1.        , 0.99992988, 0.99990585, 0.99996242, 0.99997846,
       0.99999864, 0.99999369, 0.99981358, 0.99997047, 0.99998057,
       0.99992157, 0.99982751, 0.99627859, 0.99960971, 0.99821846,
       0.99741055, 0.99777969, 0.99647939, 0.99570687, 0.99945917,
       0.99867518, 0.99770002, 0.99695035, 0.99654694])